In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.0'

# Classifying traffic conditions or hazards: a multi-class classification example


In [4]:
import os, shutil, random

In [5]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = r'./trafficnet_dataset_v1'


train_dir = os.path.join(original_dataset_dir, 'train')
validation_dir = os.path.join(original_dataset_dir, 'validation')
test_dir = os.path.join(original_dataset_dir, 'test')
os.mkdir(validation_dir)

accident_train_dir = os.path.join(train_dir, 'accident')
dense_traffic_train_dir = os.path.join(train_dir, 'dense_traffic')
fire_train_dir = os.path.join(train_dir, 'fire')
sparse_traffic_train_dir = os.path.join(train_dir, 'sparse_traffic')

accident_test_dir = os.path.join(test_dir, 'accident')
dense_traffic_test_dir = os.path.join(test_dir, 'dense_traffic')
fire_test_dir = os.path.join(test_dir, 'fire')
sparse_traffic_test_dir = os.path.join(test_dir, 'sparse_traffic')

# Directory with our validation accident pictures
validation_accident_dir = os.path.join(validation_dir, 'accident')
os.mkdir(validation_accident_dir)
validation_dense_traffic_dir = os.path.join(validation_dir, 'dense_traffic')
os.mkdir(validation_dense_traffic_dir)
validation_fire_dir = os.path.join(validation_dir, 'fire')
os.mkdir(validation_fire_dir)
validation_sparse_traffic_dir = os.path.join(validation_dir, 'sparse_traffic')
os.mkdir(validation_sparse_traffic_dir)



In [6]:
print('total training accident images:', len(os.listdir(accident_train_dir)))
print('total training dense traffic images:', len(os.listdir(dense_traffic_train_dir)))
print('total training fire images:', len(os.listdir(fire_train_dir)))
print('total training sparse traffic images:', len(os.listdir(sparse_traffic_train_dir)))
print("\n")
print('total test accident images:', len(os.listdir(accident_test_dir)))
print('total test dense traffic images:', len(os.listdir(dense_traffic_test_dir)))
print('total test fire images:', len(os.listdir(fire_test_dir)))
print('total test sparse traffic images:', len(os.listdir(sparse_traffic_test_dir)))

total training accident images: 900
total training dense traffic images: 900
total training fire images: 900
total training sparse traffic images: 901


total test accident images: 200
total test dense traffic images: 200
total test fire images: 200
total test sparse traffic images: 200


In [7]:
#Taking a 80:20 split of training and validation data

validation_accident_len = int(len(os.listdir(accident_train_dir)) * 0.2)
print('validation_accident_len', validation_accident_len)

validation_dense_traffic_len = int(len(os.listdir(dense_traffic_train_dir)) * 0.2)
print('validation_dense_traffic_len', validation_dense_traffic_len)
validation_fire_len = int(len(os.listdir(fire_train_dir)) * 0.2)
print('validation_fire_len', validation_fire_len)
validation_sparse_traffic_len = int(len(os.listdir(sparse_traffic_train_dir)) * 0.2)
print('validation_sparse_traffic_len', validation_sparse_traffic_len)




validation_accident_len 180
validation_dense_traffic_len 180
validation_fire_len 180
validation_sparse_traffic_len 180


In [8]:
# Adding images to validation directory
training_accident_files = os.listdir(accident_train_dir)
training_dense_traffic_files = os.listdir(dense_traffic_train_dir)
training_fire_files = os.listdir(fire_train_dir)
training_sparse_traffic_files = os.listdir(sparse_traffic_train_dir)
#print( random.sample(training_accident_files,validation_accident_len ))


for accident_img_file in random.sample(training_accident_files,validation_accident_len ):
    srcFile = os.path.join(accident_train_dir,  accident_img_file)
    shutil.move(srcFile,validation_accident_dir )
    
for dense_traffic_img_file in random.sample(training_dense_traffic_files,validation_dense_traffic_len ):
    srcFile = os.path.join(dense_traffic_train_dir,  dense_traffic_img_file)
    shutil.move(srcFile,validation_dense_traffic_dir )
    
for fire_img_file in random.sample(training_fire_files,validation_fire_len ):
    srcFile = os.path.join(fire_train_dir ,  fire_img_file)
    shutil.move(srcFile,validation_fire_dir )
    
for sparse_traffic_img_file in random.sample(training_sparse_traffic_files,validation_sparse_traffic_len ):
    srcFile = os.path.join(sparse_traffic_train_dir,  sparse_traffic_img_file)
    shutil.move(srcFile,validation_sparse_traffic_dir )  


In [38]:
print('total validation accident images:', len(os.listdir(validation_accident_dir)))
print('total validation dense traffic images:', len(os.listdir(validation_dense_traffic_dir)))
print('total validation fire images:', len(os.listdir(validation_fire_dir)))
print('total validation sparse traffic images:', len(os.listdir(validation_sparse_traffic_dir)))

total validation accident images: 180
total validation dense traffic images: 180
total validation fire images: 180
total validation sparse traffic images: 180


## Building our network


In [67]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(150, 150, 3)))
model.add(layers.Dense(512, activation='tanh'))
model.add(layers.Dense(128, activation='tanh'))
model.add(layers.Dense(4, activation='softmax'))

In [68]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               34560512  
_________________________________________________________________
dense_17 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 516       
Total params: 34,626,692
Trainable params: 34,626,692
Non-trainable params: 0
_________________________________________________________________


Compilation, loss function and optimiser selection

In [69]:
from keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Data preprocessing


In [70]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)
validate_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=30,
        class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=30,
        class_mode='categorical')

Found 2880 images belonging to 4 classes.
Found 720 images belonging to 4 classes.


In [71]:

for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (30, 150, 150, 3)
labels batch shape: (30, 4)


## Model Fitting

In [73]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=96,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=24)

Epoch 1/30
96/96 [==============================] - 39s 409ms/step - loss: 1.2603 - accuracy: 0.4035 - val_loss: 1.0408 - val_accuracy: 0.4111
Epoch 2/30
96/96 [==============================] - 41s 429ms/step - loss: 1.2452 - accuracy: 0.4247 - val_loss: 1.4301 - val_accuracy: 0.3125
Epoch 3/30
96/96 [==============================] - 41s 424ms/step - loss: 1.2378 - accuracy: 0.4160 - val_loss: 1.1878 - val_accuracy: 0.4444
Epoch 4/30
96/96 [==============================] - 38s 397ms/step - loss: 1.2192 - accuracy: 0.4399 - val_loss: 1.2655 - val_accuracy: 0.3611
Epoch 5/30
96/96 [==============================] - 41s 422ms/step - loss: 1.2018 - accuracy: 0.4469 - val_loss: 1.2418 - val_accuracy: 0.4194
Epoch 6/30
96/96 [==============================] - 42s 442ms/step - loss: 1.1934 - accuracy: 0.4490 - val_loss: 1.1349 - val_accuracy: 0.4264
Epoch 7/30
96/96 [==============================] - 40s 414ms/step - loss: 1.1966 - accuracy: 0.4417 - val_loss: 1.1218 - val_accuracy: 0.4458

In [ ]:
model.save('PredictingTrafficConditionsHazards_convLayerIncrease.h5')

## Plotting Training and validation Accuracy and loss

In [ ]:
tester_datagen = ImageDataGenerator(rescale=1./255)

test_generator = tester_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=1,
        class_mode='categorical',
        shuffle = 'False')

results = model.evaluate_generator(test_generator,workers=6,steps=800 )

In [ ]:
print(results)

In [ ]:
model.save('PredictingTrafficConditionsHazards_2.h5')